# Kubernetes

## 1. MSA (monolithic Architecture)

Original way:
dis: 1. 하나의 통으로 2.
### solution : scale out
how about scale up?

1. what is micro service?
- 독립적 run 가능, 독립적 배포, 의존성 최소화
<br></br>

__pro__
- scale

__con__
- 통신비용 Latency 증가
- 테스트 복잡도 증가
- 데이터 관리 어려움
<br></br>

## 2. Docker Runtime
- CRI-O, container runtime
<br></br>

## 3. Features of Kubernetes
##### __상태관리__ : 
- deployment: pod 상태관리 자동
- pod로 올리는 경우: Node 가 죽었을때 자동으로 안띄어줌<br>

##### __스케줄링__
##### __클러스터__ : 가상 네트워크를 통해 통신하기 떄문에 하나의 서버처럼 관리할수있다.
##### __서비스 디스커버리__ : 서비스간의 통신가능
<br></br>

## 4. Architecture
Master Node
- API Server (kubectl)
-- 권한관리, etcd 데이터 조회
- Etcd
-- 분산형 key/value 오픈소스 Storage
- kube-controller-manager
- kube-scheduller

Worker Node
 - kubelet
 - kube-proxy

# Kubernetes Objects

#### Pods
- Pod 단위로 스케줄링, 로드 밸런싱 가능

실전 
- kubectl get namespace -> Namespace는 하나의 프로젝트 단위, kernel 단위와 namespace와는 다름.

어떻게 Pod를 올릴까?
- exec yaml -> yaml 작성방법

Yaml 총 4가지 종류
api, kind, metadata spec

In [ ]:
apiVersion: v1 (for deployment its apps/v1)
kind: Pod 
metadata: 
    namespace: study
    name: knk-pod
spec:
    containers:
        -image: nginx
         name: nginx
            
:set paste

In [ ]:
kubectl get pod -n study
kubectl get rs
kubectl get deployment
kubectl get all -n study

In [ ]:
watch kubectl get pod -n study (show live)
describe pod 

In [ ]:
kubectl create -f nsk.pod.yaml (file 읽어서 실행)
kubectl run help
kubectl run nginx --image=nginx (pod or deployment)
kubectl run nginx --image=nginx --dry-run client -o yaml (pod 실행, yaml 출력) 출력후 resource, strategy, status는 빼야함
kubectl get po
kubectl get ns (namespace)
kubectl get all


Pod 접근 <br>
`kubectl exec -n study ljh-pod(or podName or ID) -it -- bash`
- log 확인
- process, 상태확인.
- 파일 옮길때.

In [ ]:
replicaset : control manager의 기능
but deployment 를 사용함 = replicaset의 상위호환 왜냐? RollBack, Update 기능을 지원하기 때문.
    selector: 사용하고 싶은 pod를 묶으주는 역활

In [ ]:
apiVersion: apps/v1
kind: ReplicaSet
metadata:
    name: psk-rs
    namespace: study
    labels:
     a: b  (rs, pod아무거나)
     tier: frontend
     app: nginx
spec: (replicaset에 대한 spec)
    replicas: 3
    selector:
     matchLabels:
       tier: frontend
    template:
     metadata:
      labels:
       tier: frontend
    spec:
     containers:
     - image: nginx
       name: nginx

In [ ]:
kube label pods -n study psk-rs-id
kubectl edit deployments -n study

In [ ]:
rollingupdate vs recreate (자료가 부족할때, 시간대비 업데이트 필요)

In [ ]:
서비서: k8s 파드에있는 포트나 지정을 해주고 서비스에서 지정을 해주고 연결해서 외부접근가능
    
    1. clusterIp - 클러스터 내부
    2. NodePort - 노드의 특정 포인트를 사용하여 접근
    3. LoadBalancer
    4. External Name

In [ ]:
kubectl get deployment psk-rs -n study -o yaml > back.yaml

In [ ]:
ports

apiVersion: v1 (for deployment its apps/v1)
kind: Pod 
metadata: 
    namespace: study
    name: knk-pod
spec:
    containers:
        -image: nginx
         name: nginx
         ports:
         - containerPort: 80
            
:set paste


Server를 사용해 ClusterIP로 파드끼리 연결해주는방법 : 내부에서 통신이 가능하기때문에 외부에서 접근하게 쓰려면 Proxy사용

In [ ]:
apiVersion: v1
kind: Service
metadata: 
  name: knk-service
  namespace: study
spec:
  type: ClusterIP
  selector:
    app: nginx
  ports:
  - name:http
    port: 80
    targetPort: 80
    protocol: TCP

In [ ]:
endpoints를 get po를 실행해 연결이 됫는지 안됬는지 봐야함


apiVersion: v1
kind: Service
metadata: 
  name: knk-service
  namespace: study
spec:
  type: ClusterIP
  selector:
    app1: nginx1 ############
  ports:
  - name:http
    port: 80
    targetPort: 80
    protocol: TCP
        

        kubectl describe svc app-service -n service

In [ ]:
deployment
replicas:3
name: 아무거나
port: 80 으로 띄워서 서비스에 연결해서 엔드포인트가 연결되게


In [ ]:
kubectl get deployments psk-rs -n study -o yaml > 파일이름.yaml
kubectl delete po psk-port-pod -n studyvi
kubectl describe svc app-service -n study
kubectl get all -n study
kubectl get pods -n assignment --show-labels
kubectl label pods -n study psk-rs-id
kubectl get po -o wide -n assignment

In [ ]:
two connected

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: knk-rs
  namespace: assignment
spec:
  replicas: 3
  selector:
    matchLabels:
      app1: nginx1
  template:
    metadata:
      labels:
        app1: nginx1
    spec:
      containers:
      - image: nginx
        name: nginx


In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: knk-svc
  namespace: assignment
spec:
  type: ClusterIP
  selector:
    app1: nginx1
  ports:
  - name: http
    port: 80
    targetPort: 80
    protocol: TCP

### Nodeport -> 1 ~ 6까지 있는데 외부에서 접근가능 ClusterIP는 내부끼리 접근 가능

포트당 하나의 서비스 = 30000 - 32767 범위내 포트

## Example

In [ ]:
### FOR POD
apiVersion: v1
kind: Pod
metadata:
  name: knk-port-pod
  namespace: study
  labels:
    app: knk-nginx
spec:
  containers:
  - image: nginx
    name: nginx
    ports:
    - containerPort: 80

In [ ]:
### FOR CLUSTERIP
apiVersion: v1
kind: Service
metadata:
  name: knk-service
  namespace: study
spec:
  type: ClusterIP
  selector:
    app: knk-nginx
  ports:
  - name: http
    port: 80
    targetPort: 80
    protocol: TCP

In [ ]:
### FOR NODEPORT
apiVersion: v1
kind: Service
metadata:
  name: knk-nodeport-service
  namespace: study
spec:
  type: NodePort
  selector:
    app: knk-nginx
  ports:
  - name: http
    port: 80
    targetPort: 80
    protocol: TCP

In [ ]:
then now try 192.168.201.231 ~ 232 : node_port_number

# LoadBalancer (Nodeport 타입의 확장판)

- Nodeport 와는 다르게 External-IP 를 제공한다.

Nodeport vs External-IP ?
- Nodeport는 CluterIP에서 nodeport를 추가해서 접근하고 LB는 할당된 IP로 접근하고 